In [13]:
import svgwrite
from svgwrite import rgb
from svgwrite import cm, mm   
from colorsys import hsv_to_rgb
import numpy as np
import itertools
from joblib import Parallel, delayed

from IPython.display import SVG, display

In [14]:
formulae = [[0, 1, 1], [1, 1, 1], [2, 0, 1], [3, 2, 1], [2, 3, 0]]
moduli = [2, 3, 5, 7]

In [39]:
field_modulus = 3
max_time = 100
max_coordinate = max_time
radius = 1.2
gap_x = 3
gap_y = 3
offset_x = 2 + gap_x * max_coordinate
offset_y = 2
multiplier = 150 / max_time
stroke_width = .2
line_width = .9
line_opacity = .1
min_opacity = .1
random_initialization = True

parameters = {
    'field_modulus' : 3,
    'max_time' : 100,
    'max_coordinate' : max_time,
    'radius' : 1.2,
    'gap_x' : 3,
    'gap_y' : 3,
    'offset_x' : 2 + gap_x * max_coordinate,
    'offset_y' : 2,
    'multiplier' : 150 / max_time,
    'stroke_width' : .2,
    'line_width' : .9,
    'line_opacity' : .1,
    'min_opacity' : .1,
    'random_initialization' : True,
}

In [40]:
def get_coordinates(x, time):
    return (multiplier * (offset_x + x * gap_x), multiplier * (offset_y + time * gap_y))

def make_figure(file_name, current_position_indicators, field_modulus, formula, parameters):
    field_modulus = parameters['field_modulus']
    max_time = parameters['max_time']
    max_coordinate = parameters['max_coordinate']
    radius = parameters['radius']
    gap_x = parameters['gap_x']
    gap_y = parameters['gap_y']
    offset_x = parameters['offset_x']
    offset_y = parameters['offset_y']
    multiplier = parameters['multiplier']
    stroke_width = parameters['stroke_width']
    line_width = parameters['line_width']
    line_opacity = parameters['line_opacity']
    min_opacity = parameters['min_opacity']
    random_initialization = parameters['random_initialization']
     
    dwg = svgwrite.Drawing(file_name + '.svg', size=(u'970', u'500'))
    for time in range(max_time + 1):
        for x in range(-max_coordinate, max_coordinate):
            current_value = current_position_indicators[x]
                # print(get_coordinates(x, time), get_coordinates(x, time + 1))
            if current_value % field_modulus != 0 and time != max_time:
                for delta_x, formula_multiplier in zip([-1, 0, 1], formula):
                    line_width_scalar = ((formula_multiplier * current_value) % field_modulus) / field_modulus
                    dwg.add(
                        dwg.line(
                            start=get_coordinates(x, time),
                            end = get_coordinates(x + delta_x, time + 1),
                            stroke='black',
                            stroke_opacity=line_opacity,
                            stroke_width=multiplier * line_width_scalar * line_width))
            if current_value:
                if current_value % field_modulus != 0:
                    actual_current_value = current_value % field_modulus
                    fill_color = rgb(*[256 * x for x in hsv_to_rgb(0.0, 0.70, 0.90)])
                    fill_opacity = min_opacity + (actual_current_value + 1) * (1.0 - min_opacity) / field_modulus
                    stroke_opacity = fill_opacity
                    stroke_width_multiplier = (actual_current_value + 1) / field_modulus
                elif current_value != 0:
                    fill_color = rgb(*[256 * x for x in hsv_to_rgb(0.6, 0.70, 0.70)])
                    fill_opacity = line_opacity
                    stroke_opacity = line_opacity
                    stroke_width_multiplier = 1 / field_modulus
                dwg.add(
                    dwg.circle(
                        center=get_coordinates(x, time),
                        r=multiplier * radius,
                        fill=fill_color,
                        fill_opacity=fill_opacity,
                        stroke='black',
                        stroke_opacity=stroke_opacity,
                        stroke_width=multiplier * stroke_width_multiplier * stroke_width))
                
        if time == max_time:
            break

        current_position_indicators = [x % field_modulus for x in current_position_indicators]
        new_position_indicators = [0 for _ in range(2 * max_coordinate)]

        for position, current_value in enumerate(current_position_indicators):
            for delta_x, formula_multiplier in zip([-1, 0, 1], formula):
                position_to_be_updated = (position + delta_x) % len(new_position_indicators)
                new_position_indicators[position_to_be_updated] += formula_multiplier * current_value
                # new_position_indicators[position] += current_value
                # new_position_indicators[(position + 1) % len(new_position_indicators)] += current_value
        current_position_indicators = [x for x in new_position_indicators]
            
    return dwg

In [41]:
def generate_svg(formula, field_modulus, random_initialization, parameters):
    current_position_indicators = [
        np.random.randint(field_modulus) if random_initialization else 0
        for _ in range(2 * max_coordinate)
    ]
    if not random_initialization:
        current_position_indicators[0] = 1

    current_formula = [x % field_modulus for x in formula]
    file_name = ''.join([
        'random' if random_initialization else 'single',
        '_levels_',
        str(max_time),
        '_sum_of_',
        '_'.join([str(x) for x in current_formula]),
        '_mod_',
        str(field_modulus),
    ])

    dwg = make_figure(file_name, current_position_indicators, field_modulus, current_formula, parameters)
    print(file_name)
    SVG(dwg.tostring())
    dwg.save()

In [42]:
Parallel(n_jobs=-1)(
    delayed(generate_svg)(formula, field_modulus, random_initialization=True, parameters=parameters)
    for formula, field_modulus in itertools.product(*[formulae, moduli]))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [32]:
generate_svg(formula, field_modulus, random_initialization=False, parameters=parameters)

single_levels_100_sum_of_1_1_1_mod_3
